# SoundLapse

In [1]:
# -*- coding: utf-8 -*-
''' TimeLapse 2019 '''
import Soundlapse.funcionestimelapse as tl
from Soundlapse import logspec
from Soundlapse import file
import numpy as np
import os
import subprocess
import sys
import soundfile as audio
from scipy.io import wavfile
import IPython.display as ipd
print ('Iniciando Sound-Lapse.')

[INFO   ] [Logger      ] Record log in /Users/Nigglea/.kivy/logs/kivy_19-11-29_0.txt
[INFO   ] [Kivy        ] v2.0.0.dev0, git-Unknown, 20191112
[INFO   ] [Kivy        ] Installed at "/usr/local/lib/python3.7/site-packages/kivy/__init__.py"
[INFO   ] [Python      ] v3.7.5 (default, Nov  1 2019, 02:16:38) 
[Clang 10.0.0 (clang-1000.11.45.5)]
[INFO   ] [Python      ] Interpreter at "/usr/local/opt/python/bin/python3.7"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [ImageLoaderFFPy] Using ffpyplayer 4.2.0
[INFO   ] [Image       ] Providers: img_tex, img_imageio, img_dds, img_sdl2, img_pil, img_ffpyplayer, img_gif 
[INFO   ] [Text        ] Provider: sdl2


Iniciando Sound-Lapse.


## Ingreso de archivo(s) de audio
#### Seleccionar audio(s) que compondran la capsula de audio

In [2]:
filez = file.filebrowser()

## Inputs
#### Ingreso de parametros deseados para la creacion de la muestra representativa de audio

In [3]:
in_t_inic      = float(input('Comienzo de Sounlapse:[min] ') )  # min. Tiempo en que comienza el timelapse.
in_t_segm      = float(input('Duracion de cada muestra:[seg] ') )  # seg. Duración de cada segmento.
in_t_delta     = float(input('Intervalo entre muestras:[min] '))  # min. Delta de tiempo entre segmentos.
in_t_fade      = float(input('Duracion de entrelazamineto:[seg] '))  # seg. Duración de fades y crossfaders.
crossfade_type = int(input('Tipo de entrelazamiento:(1 Lineal,2 Exponencial,3 Logaritmico) '))  # Tipo de Crossfade.
cross_out      = tl.get_cross_out(crossfade_type)  # Obtiene el dato de tipo de crossfade.

Comienzo de Sounlapse:[min] 10
Duracion de cada muestra:[seg] 22
Intervalo entre muestras:[min] 60
Duracion de entrelazamineto:[seg] 6
Tipo de entrelazamiento:(1 Lineal,2 Exponencial,3 Logaritmico) 3
Ha escogido fade logarítmico.


## Creacion SoundLapse

In [4]:
chunk_fadeout = np.zeros(shape=(1, 2))
cont          = 0  # Conteo. Para registro de barra de progreso.
for filepath in filez:
    if filepath == filez[0]:
        time_lapse_vector = np.zeros(shape=(1, 2))  # Prealocación de vector de ceros para el timelapse.
    in_sig                                  = audio.SoundFile(filepath, 'r')  # Lectura de Audio.
    samp_freq                               = in_sig.samplerate  # Frecuencia de Muestreo.
    len_in_sig                              = len(in_sig)  # Número de muestras de la señal.
    # Preprocesado: Transformación de seg/min a muestras.
    m_desc, m_fades, m_inic_desc, m_segment = tl.time_to_samples(in_t_inic, in_t_segm, in_t_delta, in_t_fade, samp_freq)
    fadein_vect, fadeout_vect, cross_out    = tl.get_crossfaders(crossfade_type, m_fades)  # Vectores de crossfaders.
    # Segmentación
    n_chunks                                = tl.get_nchunks(len_in_sig, m_inic_desc, m_segment, m_desc)  # Número de segmentos del timelapse
    m_rest                                  = len_in_sig  # Muestras restantes del vector de audio original.
    time_lapse_vector, chunk_fadeout, cont  = tl.get_timelapse_one_file(in_sig, n_chunks, fadein_vect, fadeout_vect,
                                                                       m_inic_desc, m_segment, m_desc, m_fades,
                                                                       time_lapse_vector, filez, filepath,
                                                                       chunk_fadeout, cont)  # Genera TimeLapse

print ("Ha finalizado la segmentación de audio.")

Progreso: 99.99999999999999%.  Ha finalizado la segmentación de audio.


## Exportacion 
#### Se guarda archivo en formato de audio

In [6]:
outfile = 'Outputs/SoundLapse_'+str(in_t_inic)+'_'+str(in_t_segm)+'_'+str(in_t_delta)+'_'+str(in_t_fade)+'_'+'cross_' + cross_out
f       = audio.SoundFile(outfile, 'w',samplerate=samp_freq,channels=2,subtype='PCM_16',endian='BIG',format='WAV')
f.write(time_lapse_vector)
f.close()
print ("Exportando archivo " + outfile + ".wav")

Exportando archivo Outputs/SoundLapse_10.0_22.0_60.0_6.0_cross_logaritmico.wav


### Estereo a mono
#### Para la representacion grafica del Soundlapse

In [7]:
sample_rate, samples = wavfile.read(outfile)
mono_samples         = tl.stereoToMono(samples)
f_mono               = audio.SoundFile(outfile+"_mono.wav", 'w',samplerate=samp_freq,channels=1,subtype='PCM_16',endian='BIG',format='WAV')
f_mono.write(mono_samples)
f_mono.close()
print ('TimeLapse finalizado.')

TimeLapse finalizado.
